In [ ]:
#Code based on TWO API databases, NewsAPI and NewsData. There are around 100 requests a day for news API and 200 for NewsData
#Code mergers both results into one output, sorted BY RELAVANCY 

import requests

NEWSAPI_KEY = '5581d8d29db643f6893fe91896b724ac'#PRIVATE API KEY for NEWSAPI.COM
NEWSDATA_KEY = 'pub_6184041c186d74171dc31ec270cc41cba5695' #PRIVATE API KEY for NEWSDATA.IO
NEWSAPI_URL = 'https://newsapi.org/v2/everything'
NEWSDATA_URL = 'https://newsdata.io/api/1/news'

# Prompt the user for company name and keyword(s)
company = input("Enter the company you want to search for: ").strip()
user_keywords = input("Enter the keyword(s) you want to filter by (comma-separated): ").strip()

keywords = [kw.strip().lower() for kw in user_keywords.split(',') if kw.strip()]

newsapi_params = {
    'q': company,  
    'language': 'en',
    'sortBy': 'relevance',
    'apiKey': NEWSAPI_KEY,
}

newsdata_params = {
    'q': company,
    'language': 'en',
    'apiKey': NEWSDATA_KEY
}

def fetch_newsapi_articles():
    try:
        response = requests.get(NEWSAPI_URL, params=newsapi_params)
        response.raise_for_status()
        articles = response.json().get('articles', [])
        normalized_articles = []
        for art in articles:
            normalized_articles.append({
                'title': art.get('title', ''),
                'url': art.get('url', ''),
                'description': art.get('description', ''),
                'content': art.get('content', '')
            })
        return normalized_articles
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching from NewsAPI: {e}")
        return []

def fetch_newsdata_articles():
    try:
        response = requests.get(NEWSDATA_URL, params=newsdata_params)
        response.raise_for_status()
        data = response.json()
        articles = data.get('results', [])
        normalized_articles = []
        for art in articles:
            normalized_articles.append({
                'title': art.get('title', ''),
                'url': art.get('link', ''),
                'description': art.get('description', ''),
                'content': art.get('content', '')
            })
        return normalized_articles
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching from Newsdata.io: {e}")
        return []

def fetch_and_filter_articles():

    newsapi_articles = fetch_newsapi_articles()
    newsdata_articles = fetch_newsdata_articles()
    combined_articles = newsapi_articles + newsdata_articles

    unique_articles = []
    seen_urls = set()
    for art in combined_articles:
        if art['url'] and art['url'] not in seen_urls:
            seen_urls.add(art['url'])
            unique_articles.append(art)

    filtered_articles = []
    for article in unique_articles:
        content = (article.get('content', '') or '') + ' ' + (article.get('description', '') or '')
        content = content.lower()
        if any(keyword in content for keyword in keywords):
            filtered_articles.append(article)

    if filtered_articles:
        print('\nFiltered Articles on', company, ":\n")
        for idx, article in enumerate(filtered_articles, 1):
            print(f"Article {idx}:")
            print(f"Title: {article['title']}")
            print(f"URL: {article['url']}")
            print(f"Description: {article.get('description', 'No description available.')}\n")
    else:
        print("\nNo articles found matching the specified keywords for this company.")

fetch_and_filter_articles()

#RETURN FORMAT IS A LIST OF ARTICLES SORTED BY 'RELEVANCY'. 
#OUTPUT ARE NOT COMPLETE, PAYWALLED NEWS SOURCES WILL NOT BE QUERYED. 

Enter the company you want to search for:  CANADIAN NATURAL RESOURCES
Enter the keyword(s) you want to filter by (comma-separated):  Gas



Filtered Articles on CANADIAN NATURAL RESOURCES :

Article 1:
Title: Atria Investments Inc Lowers Position in Canadian Natural Resources Limited (NYSE:CNQ)
URL: https://www.etfdailynews.com/2024/11/18/atria-investments-inc-lowers-position-in-canadian-natural-resources-limited-nysecnq/
Description: Atria Investments Inc trimmed its holdings in shares of Canadian Natural Resources Limited (NYSE:CNQ – Free Report) (TSE:CNQ) by 14.5% during the 3rd quarter, Holdings Channel reports. The fund owned 26,878 shares of the oil and gas producer’s stock after sel…

Article 2:
Title: BNP Paribas Financial Markets Boosts Stake in Canadian Natural Resources Limited (NYSE:CNQ)
URL: https://www.etfdailynews.com/2024/12/04/bnp-paribas-financial-markets-boosts-stake-in-canadian-natural-resources-limited-nysecnq/
Description: BNP Paribas Financial Markets raised its stake in Canadian Natural Resources Limited (NYSE:CNQ – Free Report) (TSE:CNQ) by 237.5% during the third quarter, Holdings Channel reports